In [102]:
# imports
from copy import copy, deepcopy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import linear_model
import pandas as pd 
import numpy as np
from sklearn import metrics  
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder


%matplotlib inline

In [103]:
# Load the Wine Dataset
df_wine = pd.read_csv("df_wine_horizontal_tests.csv", encoding = 'utf8', index_col=0)
df_wine = df_wine.reset_index()
# check data
df_wine.head(4)

,index,country,province,region,price,variety,points,year_of_wine
0,0,US,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013
1,1,US,Michigan,Lake Michigan Shore,13.0,Riesling,87,2013
2,2,US,Oregon,Willamette Valley,65.0,Pinot Noir,87,2012
3,3,Spain,Northern Spain,Navarra,15.0,Tempranillo-Merlot,87,2011


In [104]:
df_wine.head(4)

,index,country,province,region,price,variety,points,year_of_wine
0,0,US,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013
1,1,US,Michigan,Lake Michigan Shore,13.0,Riesling,87,2013
2,2,US,Oregon,Willamette Valley,65.0,Pinot Noir,87,2012
3,3,Spain,Northern Spain,Navarra,15.0,Tempranillo-Merlot,87,2011


In [105]:
# check distinct countries
countries = pd.unique(df_wine['country'])
print(countries)

['US' 'Spain' 'Italy' 'France' 'Argentina' 'Australia' 'Canada']


In [106]:
# todo: check frequencies of country
frequencies = df_wine['country'].value_counts()
frequencies.sort_values
print(frequencies)

US           53116
France       16319
Italy        16069
Spain         6059
Argentina     3664
Australia     2223
Canada         253
Name: country, dtype: int64


In [107]:
name = frequencies.index[0]
value = frequencies[0]
print("Most frequent: {name} ({value})".format(name=name, value=value))

Most frequent: US (53116)


In [108]:
# ['US' 'Spain' 'Italy' 'France' 'Argentina' 'Australia' 'Canada']

us_wines = df_wine[df_wine['country'].str.match('US')] # 53116
spain_wines = df_wine[df_wine['country'].str.match('Spain')] # 16319
italy_wines = df_wine[df_wine['country'].str.match('Italy')] # 16069
france_wines = df_wine[df_wine['country'].str.match('France')] # 6565
argentina_wines = df_wine[df_wine['country'].str.match('Argentina')] # 3664
australia_wines = df_wine[df_wine['country'].str.match('Australia')] # 2223
canada_wines = df_wine[df_wine['country'].str.match('Canada')] # 253

argentina_wines.head(4)



,index,country,province,region,price,variety,points,year_of_wine
11,11,Argentina,Other,Cafayate,30.0,Malbec,87,2010
12,12,Argentina,Mendoza Province,Mendoza,13.0,Malbec,87,2011
143,145,Argentina,Other,Salta,12.0,Torrontés,88,2007
169,171,Argentina,Mendoza Province,Luján de Cuyo,22.0,Malbec,90,2014


In [109]:
# Auto encodes any dataframe column of type category or object.
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df

In [110]:
# one hot encoding
df_dummy = dummyEncode(df_wine[['country','province','region','variety']])
df_dummy.columns = ['country_num', 'province_num','region_num','variety_num']
df_wine = pd.concat([df_wine, df_dummy], axis=1)

/Users/roxana/.virtualenvs/dojo3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [111]:
df_wine.head(4)
# us country num = 6

,index,country,province,region,price,variety,points,year_of_wine,country_num,province_num,region_num,variety_num
0,0,US,Oregon,Willamette Valley,14.0,Pinot Gris,87,2013,6,40,999,311
1,1,US,Michigan,Lake Michigan Shore,13.0,Riesling,87,2013,6,27,472,340
2,2,US,Oregon,Willamette Valley,65.0,Pinot Noir,87,2012,6,40,999,315
3,3,Spain,Northern Spain,Navarra,15.0,Tempranillo-Merlot,87,2011,5,36,629,423


In [112]:
# Variety
df_variety = df_wine.variety.str.get_dummies()
df_variety.columns = ['v_' + col for col in df_variety.columns] # naming the columns
df_wine = pd.concat([df_wine, df_variety], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('variety', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('variety_num', axis=1) # drop the country attribute since different attributes per country value

In [113]:
# Country
df_country = df_wine.country.str.get_dummies() # get the pivot of the country attribte
df_country.columns = ['country_' + col for col in df_country.columns] # naming the columns
df_wine = pd.concat([df_wine, df_country], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('country', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('country_num', axis=1) # drop the country attribute since different attributes per country value

In [114]:
# Province
df_province = df_wine.province.str.get_dummies() # get the pivot of the country attribte
df_province.columns = ['province_' + col for col in df_province.columns] # naming the columns
df_wine = pd.concat([df_wine, df_province], axis=1) # combine main dataframe with country matrix dataframe
df_wine = df_wine.drop('province', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('province_num', axis=1) # drop the country attribute since different attributes per country value

In [115]:
# Region
df_wine = df_wine.drop('region', axis=1) # drop the country attribute since different attributes per country value
df_wine = df_wine.drop('region_num', axis=1) # drop the country attribute since different attributes per country value

In [116]:
df_wine.head(5)

,index,price,points,year_of_wine,v_Abouriou,v_Aglianico,v_Airen,v_Albana,v_Albanello,v_Albariño,...,province_Spanish Islands,province_Tasmania,province_Texas,province_Tuscany,province_Veneto,province_Vermont,province_Victoria,province_Virginia,province_Washington,province_Western Australia
0,0,14.0,87,2013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,13.0,87,2013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,65.0,87,2012,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,15.0,87,2011,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,16.0,87,2013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
# check new headers
headers = list(df_wine)

for header in headers:
    if "country_" in header:
        print(header)

country_Argentina
country_Australia
country_Canada
country_France
country_Italy
country_Spain
country_US


In [118]:
# validate number of wines from US, should be 53116
frequencies_us = df_wine['country_US'].value_counts()
if frequencies_us[1] == us_wines.shape[0]:
    print("[OK] Sizes match after hot encoding.")

[OK] Sizes match after hot encoding.


In [119]:
# check only numbers in data
df_wine.head(5)

,index,price,points,year_of_wine,v_Abouriou,v_Aglianico,v_Airen,v_Albana,v_Albanello,v_Albariño,...,province_Spanish Islands,province_Tasmania,province_Texas,province_Tuscany,province_Veneto,province_Vermont,province_Victoria,province_Virginia,province_Washington,province_Western Australia
0,0,14.0,87,2013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,13.0,87,2013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,65.0,87,2012,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,15.0,87,2011,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,16.0,87,2013,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
# split data
df_X = df_wine.drop('points', axis=1)
df_Y = df_wine[['points']]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=0) 

In [124]:
def get_hyperparameters(X_train, y_train):
    regressor3 = DecisionTreeRegressor()
    param_dist = {'max_depth': sp_randint(2,16),
                  'min_samples_split': sp_randint(2,16)}

    n_iter_search = 20
    clfrs = RandomizedSearchCV(regressor3, 
                               param_distributions=param_dist,
                               scoring='neg_mean_squared_error',
                               cv=5 , n_jobs=1, verbose=1,
                               n_iter=n_iter_search)
    clfrs.fit(X_train, y_train)
    params = clfrs.best_params_
    return params["max_depth"], params["min_samples_split"]

In [125]:
# get hyperparameters
max_depth, min_samples_split = get_hyperparameters(X_train, y_train)
print("max_depth: {}, min_samples_split: {}".format(max_depth, min_samples_split)) 

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.5min finished


max_depth: 8, min_samples_split: 15


# BASE RESULT

In [126]:
regressor = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split)  

# Train the model using the training sets
regressor.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regressor.predict(X_test)

# The evaluation metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Mean Absolute Error: 1.8526171286602233
Mean Squared Error: 5.488663882413888
Root Mean Squared Error: 2.3427897648773115
